In [2]:
#pip install langchain_openai

In [3]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.chains import LLMChain
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.schema import Document
from langchain.schema.runnable import RunnablePassthrough, RunnableBranch
from typing import Literal, List, Dict, Any
from pydantic import BaseModel, Field

from langchain_core.runnables import RunnableBranch
from langchain_core.prompts import PromptTemplate


# Initialize LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")


In [4]:
from dotenv import load_dotenv
load_dotenv()

True

# Sentiment classification

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

# Define the sentiment analysis response format
sentiment_parser = JsonOutputParser()

# Sentiment analysis prompt
##TODO: Write prompt, which takes customer review and returns sentiment analysis in the following format
## {"positive_sentiment": bool, "reasoning": string}

sentiment_prompt = PromptTemplate(
    template="""....""",
    input_variables=["review"]
)

# Create sentiment analysis chain - use proper configuration to get direct output
sentiment_chain = sentiment_prompt | llm | sentiment_parser


In [ ]:
# Test the sentiment analysis chain with examples
# Example 1: Positive review
positive_test = "The service was outstanding and the food was delicious. I'll definitely come back!"

##TODO: invoke sentiment chain with positive review, evaluate if it is positive and print the result
positive_sentiment = sentiment_chain.invoke({.....})

print(f"Is Positive? {positive_sentiment['positive_sentiment']}")
print(f"Reasoning: {positive_sentiment['reasoning']}")



Is Positive? True
Reasoning: The review uses positive language such as 'outstanding' and 'delicious' to describe the service and food, respectively. Additionally, the reviewer expresses a desire to return, indicating a positive experience overall.


In [10]:
positive_sentiment

{'positive_sentiment': True,
 'reasoning': "The review uses positive language such as 'outstanding' and 'delicious' to describe the service and food, respectively. Additionally, the reviewer expresses a desire to return, indicating a positive experience overall."}

In [11]:


negative_test = "The wait was too long and the staff was rude. The food was cold when it arrived."
negative_sentiment = sentiment_chain.invoke({"review": negative_test})

print(f"Is Positive? {negative_sentiment['positive_sentiment']}")
print(f"Reasoning: {negative_sentiment['reasoning']}")


Is Positive? False
Reasoning: The review expresses dissatisfaction with the experience, mentioning a long wait time, rude staff, and cold food, all of which contribute to a negative sentiment.


# Routing negative and positive reviews


## Positive review handling

In [ ]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

##TODO: prepare prompt for positive review handling, it should take the following steps:
#1. Thank user for their positive feedback
#2. Offers a voucher related to the thing they liked
#3. Encourage user to visit again
#Output should be a json with the following format {"message": str}

positive_prompt = PromptTemplate(
    template=""".....
""",
    input_variables=["review"]
)

# Use JsonOutputParser to get properly formatted JSON output
positive_chain = positive_prompt | llm | JsonOutputParser()


In [13]:
positive_message = positive_chain.invoke({"review": positive_test})
print(positive_message["message"])

Dear Customer,

Thank you so much for your wonderful feedback! We're thrilled to hear that you
found our service outstanding and enjoyed our delicious food. As a token of our
appreciation, we'd like to offer you a voucher for a complimentary dessert on
your next visit.

We look forward to welcoming you back soon!

Best regards,
The Team


## Negative review handling

In [15]:

negative_prompt = PromptTemplate(
    template="""You are a customer service representative for a company.
A customer has left a negative review about our products or services.

Customer Review: {review}

Based on the review, identify what they specifically disliked and create a personalized response that:
1. Apologizes for their negative experience
2. Addresses the specific issue they mentioned
3. Explains how you'll mitigate this issue in the future
4. Offers a 25% discount on their next visit
5. Thanks them for their feedback


Return your response as a valid JSON object with the following format:
{{"message": str}}

Make sure to add appropriate newline after approximately every 80 characters for better readability.
""",
    input_variables=["review"]
)

negative_chain = negative_prompt | llm | JsonOutputParser()

In [16]:
negative_message = negative_chain.invoke({"review": negative_test})
print(negative_message["message"])

Dear Valued Customer,

We sincerely apologize for the negative experience you had during your
recent visit. We understand that the long wait time, the rudeness of our
staff, and receiving cold food are unacceptable and not reflective of the
standards we strive to maintain.

To address these issues, we are actively working on improving our service
speed and providing additional training to our staff to ensure that all
customers are treated with the utmost respect and courtesy. We are also
reviewing our food preparation and delivery processes to ensure that all
meals are served hot and fresh.

As a token of our apology, we would like to offer you a 25% discount on
your next visit. We hope this will give us the opportunity to provide you
with the quality experience you deserve.

Thank you for your feedback, as it helps us improve our services. We look
forward to welcoming you back soon.

Warm regards,

[Your Company Name] Customer Service Team


## Combine chains

In [17]:
# Create the branching logic with RunnablePassthrough directly in the chain
##TODO Use RunnableBranch to create a chain that will invoke the sentiment analysis first and then based on the result,
# invoke either the positive or negative response chain.

full_chain = (
    RunnablePassthrough.assign(
        sentiment_result=sentiment_chain
    ) 
    | RunnableBranch(
        (
            lambda x: x["sentiment_result"]["positive_sentiment"], 
            {
                "review": lambda x: x["review"],
            } | positive_chain
        ),
        (
            lambda x: not x["sentiment_result"]["positive_sentiment"],
            {
                "review": lambda x: x["review"],
            } | negative_chain
        ),
        # Default fallback
        lambda x: {"message": f"Error: Unable to determine sentiment for: {x['review']}"}
    )
)


In [20]:
## Validate if output of full_chain is what we expect -> 25% discount for negative, voucher for positive

In [18]:

negative_message = full_chain.invoke({"review": negative_test})
print(negative_message["message"])

Dear Valued Customer,

We sincerely apologize for the negative experience you had during your
recent visit. We understand that the long wait time, the rudeness of our
staff, and the cold food were disappointing and not up to the standards
we strive to maintain.

To address these issues, we are actively working on improving our
service efficiency and providing additional training to our staff to
ensure a more pleasant and timely experience for all our customers. We
are also reviewing our food preparation processes to ensure that meals
are served hot and fresh.

As a token of our apology, we would like to offer you a 25% discount on
your next visit. We hope this will give us the opportunity to provide
you with the quality service and experience you deserve.

Thank you for your feedback, as it helps us improve and serve you
better. We look forward to welcoming you back soon.

Warm regards,

[Your Company Name] Customer Service Team


In [19]:

positive_message = full_chain.invoke({"review": positive_test})
print(positive_message["message"])

Thank you so much for your wonderful feedback! We're thrilled to hear that you 
enjoyed our outstanding service and delicious food. As a token of our appreciation, 
we'd like to offer you a voucher for a complimentary dessert on your next visit. 
We look forward to welcoming you back soon!
